In [1]:
import modules.models as cnv_model
import modules.utils as utils
import torch
from torchinfo import summary

# Setup Model

In [2]:
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

In [3]:
mbnet_v3 = cnv_model.setup_model(
    model_name="MOBILENETV3", 
    num_classes=2, 
    device=DEVICE)

shufflenet_v2 = cnv_model.setup_model(
    model_name="SHUFFLENET", 
    num_classes=2, 
    device=DEVICE)

Using MOBILENETV3 Classifier
Sequential(
  (0): Linear(in_features=576, out_features=1024, bias=True)
  (1): Hardswish()
  (2): Dropout(p=0.2, inplace=True)
  (3): Linear(in_features=1024, out_features=1000, bias=True)
)
Model Pretrained has 576 in features in last layer
Using SHUFFLE Classifier
Linear(in_features=1024, out_features=1000, bias=True)
Model Pretrained has 1024 in features in last layer


### Torch Summary

### MobileNetV3

In [4]:
print(summary(mbnet_v3, input_size=(1,3, 224, 224)))

Layer (type:depth-idx)                                  Output Shape              Param #
PRETRAINED_MODEL                                        [1, 2]                    --
├─Sequential: 1-1                                       [1, 576, 1, 1]            --
│    └─Sequential: 2-1                                  [1, 576, 7, 7]            --
│    │    └─Conv2dNormActivation: 3-1                   [1, 16, 112, 112]         (464)
│    │    └─InvertedResidual: 3-2                       [1, 16, 56, 56]           (744)
│    │    └─InvertedResidual: 3-3                       [1, 24, 28, 28]           (3,864)
│    │    └─InvertedResidual: 3-4                       [1, 24, 28, 28]           (5,416)
│    │    └─InvertedResidual: 3-5                       [1, 40, 14, 14]           (13,736)
│    │    └─InvertedResidual: 3-6                       [1, 40, 14, 14]           (57,264)
│    │    └─InvertedResidual: 3-7                       [1, 40, 14, 14]           (57,264)
│    │    └─InvertedResidu

### ShuffleNetV2

In [5]:
print(summary(shufflenet_v2, input_size=(1,3, 224, 224)))

Layer (type:depth-idx)                        Output Shape              Param #
PRETRAINED_MODEL                              [1, 2]                    --
├─Sequential: 1-1                             [1, 1024, 7, 7]           --
│    └─Sequential: 2-1                        [1, 24, 112, 112]         --
│    │    └─Conv2d: 3-1                       [1, 24, 112, 112]         (648)
│    │    └─BatchNorm2d: 3-2                  [1, 24, 112, 112]         (48)
│    │    └─ReLU: 3-3                         [1, 24, 112, 112]         --
│    └─MaxPool2d: 2-2                         [1, 24, 56, 56]           --
│    └─Sequential: 2-3                        [1, 48, 28, 28]           --
│    │    └─InvertedResidual: 3-4             [1, 48, 28, 28]           (2,400)
│    │    └─InvertedResidual: 3-5             [1, 48, 28, 28]           (1,512)
│    │    └─InvertedResidual: 3-6             [1, 48, 28, 28]           (1,512)
│    │    └─InvertedResidual: 3-7             [1, 48, 28, 28]           (1,

## Load Weights and Export ONNX

### MobileNetV3

In [6]:
weights_path = './experiments/test_v01_mobilenetv3_full_ds/weights/MOBILENETV3_classifier__best_mean_F1.pt'

In [7]:
utils.load_checkpoint(weights_path, mbnet_v3)

Loading Model. Trained during 95 epochs


95

In [8]:
mbnet_v3.cpu();

#### ONNX export

In [9]:
torch_input = torch.randn(1, 3, 224, 224)
# torch.onnx.export(model, torch_input, "MobilenetV3.onnx")

In [10]:
torch.onnx.export(
    mbnet_v3,                    # The model to be exported
    torch_input,              # The sample input tensor
    "MobilenetV3_Raspi.onnx", # The output file name
    export_params=True,       # Store the trained parameter weights inside the model file
    do_constant_folding=True, # Whether to execute constant folding for optimization
    input_names=['input'],    # The model's input names
    output_names=['output'],  # The model's output names
)

### ShuffleNetV2

In [11]:
weights_path = './experiments/test_v03_shufflenet_full_ds/weights/SHUFFLENET_classifier__best_mean_F1.pt'

In [12]:
utils.load_checkpoint(weights_path, shufflenet_v2)

Loading Model. Trained during 95 epochs


95

In [13]:
shufflenet_v2.cpu();

In [15]:
torch.onnx.export(
    shufflenet_v2,             # The model to be exported
    torch_input,               # The sample input tensor
    "ShuffleNetV2_Raspi.onnx", # The output file name
    export_params=True,        # Store the trained parameter weights inside the model file
    do_constant_folding=True,  # Whether to execute constant folding for optimization
    input_names=['input'],     # The model's input names
    output_names=['output'],   # The model's output names
)